In [1]:
# imports 
%load_ext autoreload
%autoreload 2
import schillinger.rhythm as SR
import schillinger.pitch as SP
import itertools
import random
from music21 import *
%load_ext music21.ipython21

In [2]:
#convert2d to 1d 
def convert2Dto1D(data):
    flat_array = []
    for d in data:
        if isinstance(d[0], list) or isinstance(d[0], tuple):
            for l in d:
                
                flat_array.append(l)
        else:
            flat_array.append(d)
    return flat_array

In [38]:
# notes harmony 

def arpeggiator(c, es,rhythm, duration_divider):
    chords = []
    for i, seq in enumerate(c):
                
        nota = seq
        nota.duration.quarterLength = (rhythm[es%len(rhythm)]/len(c)) * duration_divider
        chords.append(nota)

    nota = pitch.Pitch(c.pitches[0])
    
    
    nota.octave = nota.octave + 2
    nota = note.Note(nota.nameWithOctave)
    #nota.duration.quarterLength = rhythm[es%len(rhythm)] * duration_divider
    nota.duration.quarterLength = (rhythm[es%len(rhythm)]/len(c)) * duration_divider
    
   
    chords.append(nota)
    #octave adjust
    
    
    #print(chords[-1].pitch.midi - chords[-2].pitch.midi, chords[-1])
    if chords[-1].pitch.midi - chords[-2].pitch.midi >= 11:
        chords[-1].octave = chords[-1].octave -1
        #print("yes")
    ##
    nota = pitch.Pitch(c.pitches[2])
    nota.octave = nota.octave
    nota = note.Note(nota.nameWithOctave)
    #nota.duration.quarterLength = rhythm[es%len(rhythm)] * duration_divider
    nota.duration.quarterLength = (rhythm[es%len(rhythm)]/len(c)) * duration_divider
    chords.append(nota)
    return chords

def showHarmony(note_sequence, melody, rhythm, length, time_meter, duration_divider, arp=False,bass=True, drop2 = False):
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    
    s = stream.Score(id='mainScore')
    p0 = stream.Part(id='melody')
    p0.timeSignature = meter.TimeSignature(TS)
    p1 = stream.Part(id='chords')
    p1.timeSignature = meter.TimeSignature(TS)
    
    for es, sequences in enumerate(note_sequence):
        
        melody = []
        chords = []
        
        
        ### chords
        chord_seq = [sequences[0]]+sequences[1]
        print(chord_seq)
        test_chord = chord.Chord(chord_seq)
        c = chord.Chord(test_chord)
        c.root = sequences[0]
        c.duration.quarterLength = rhythm[es%len(rhythm)] * duration_divider
        #c = c.sortDiatonicAscending() # for better voice leadng but not real!
        if es % 2 == 0: # rotating clock counterclock positions
            #c = c.semiClosedPosition(inPlace=False,forceOctave=4)
            pass
        else:
            #c = c.closedPosition(inPlace=False,forceOctave=4)
            pass
        if drop2:
            try:
                c.pitches[0].octave = c.pitches[0].octave-1 # drop2
                #c.pitches[-2].octave = c.pitches[-2].octave+2 # drop4
                pass
            except:
                pass
        
        if arp:
            chords =  arpeggiator(c, es, rhythm, duration_divider)
            #chords.append(nota)
            
        else:
            chords.append(c)
            
        #melody
        # get chord durations and apply on melody (for test purpose)
        dur = 0
        for d in chords:
            dur = dur + d.duration.quarterLength
        #print(dur)
        nota = pitch.Pitch(sequences[0])
        nota.octave = nota.octave
        nota = note.Note(nota.nameWithOctave)
        nota.duration.quarterLength = dur
        melody.append(nota)
        
        
        
        p0.append(melody)

        p1.append(chords)
    
    #s.insert(0, p0)
    s.insert(0, p1)
    s = s.transpose(0)
    return s


In [27]:
# MAIN FUNCTION
def harmonize_dt_bass(scales, diatonic_cycles_string, coeficients):
    SPG = SP.PitchGroup()
    bass = []
    harmonies = []
    for s in scales:
        temp_scale = [str(p) for p in s.pitches[:-1]]
        scale_expansion = SPG.expansions(temp_scale)
        C7 = scale_expansion[0]
        C3 = scale_expansion[4]
        C5 = scale_expansion[2]
        diatonic_cycles = []
        for i in diatonic_cycles_string:
            if i == "C3":
                diatonic_cycles.append(C3)
            elif i == "C5":
                diatonic_cycles.append(C5)
            elif i == "C7":
                diatonic_cycles.append(C7)
            else:
                print("diatonic_cycles_string Value Error!")
                
        diatonic_cycle = SP.pitch_cicles(temp_scale, diatonic_cycles, [1,1,1])
        harmonies.append(temp_scale)
        bass.append(diatonic_cycle)
    return [bass, harmonies]

hm = scale.HarmonicMinorScale("C4")
do = scale.DorianScale("C4")
ms = scale.MixolydianScale("C4")
ls = scale.LydianScale("C4")

hb = harmonize_dt_bass([hm,do], ["C5","C3","C7"], [2,1,2])

bass = hb[0]
scales = hb[1]
seq_rhythm = [1]
print(bass,scales)

[['C4', 'F4', 'D4', 'E-4', 'A-4', 'F4', 'G4', 'C4', 'A-4', 'B4', 'E-4', 'C4', 'D4', 'G4', 'E-4', 'F4', 'B4', 'G4', 'A-4', 'D4', 'B4', 'C4'], ['C4', 'F4', 'D4', 'E-4', 'A4', 'F4', 'G4', 'C4', 'A4', 'B-4', 'E-4', 'C4', 'D4', 'G4', 'E-4', 'F4', 'B-4', 'G4', 'A4', 'D4', 'B-4', 'C4']] [['C4', 'D4', 'E-4', 'F4', 'G4', 'A-4', 'B4'], ['C4', 'D4', 'E-4', 'F4', 'G4', 'A4', 'B-4']]


In [39]:
# music21 converter 

########################################################
#notes_permutations = SPG.melodic_forms_definition(notes)
voice_amount = 4 # for now!
theme_expansion = 0
scale_expansion_amount = 1 # # 1st order expansion for harmonizing is according to schillingers default
#print(scales)
#whole_seq = SP.get_whole_sequence(melody, scales, voice_amount, theme_expansion, scale_expansion_amount)
whole_seq = SP.get_whole_sequence_bass(bass, scales, voice_amount, theme_expansion, scale_expansion_amount)
merged_seq = list(itertools.chain(*whole_seq))
print(whole_seq, merged_seq)
## apply original melody (correct octaves)
for i, m in enumerate(merged_seq):
    #merged_seq[i][0] = merged_seq[i][0] + str(pitches[i].octave)
    pass
    
#s = showMultipleScore_MelodyHarmony(merged_seq, myscale, seq_rhythm, len(merged_seq)*2, [4,4], 2, arp=False, bass=False)

s = showHarmony(merged_seq, "myscale", seq_rhythm, len(merged_seq)*2, (4,4), 2, arp=True, bass=False, drop2=True)



#s.show('musicxml')

[[[['C4', 'E-4', 'G4', 'B4'], ['F4', 'A-4', 'C4', 'E-4']]], [[['C4', 'E-4', 'G4', 'B-4'], ['F4', 'A4', 'C4', 'E-4']]]] [[['C4', 'E-4', 'G4', 'B4'], ['F4', 'A-4', 'C4', 'E-4']], [['C4', 'E-4', 'G4', 'B-4'], ['F4', 'A4', 'C4', 'E-4']]]
[['C4', 'E-4', 'G4', 'B4'], 'F4', 'A-4', 'C4', 'E-4']


ChordException: Could not process input argument ['C4', 'E-4', 'G4', 'B4']